In [1]:
import sys
sys.path.append('../../src')

from arcgis.geometry import Point
import pandas as pd

from geoai_retail import analysis

In [2]:
origins = r'D:\projects\geoai-retail\data\raw\raw.gdb\block_groups'
origin_id_field = r'ID'
destinations = r'D:\projects\geoai-retail\data\raw\raw.gdb\coffee'
destination_id_field = r'LOCNUM'
closest_table = 'D:\projects\geoai-retail\data\interim\closest.csv'
new_destination = Point({"x":-122.6950004421445,"y":45.577250341602408,"spatialReference":{"wkid":4326,"latestWkid":4326}})
remove_destination_id = 705626027
origin_weighting_points = None

In [3]:
%%time
origin_df_add = analysis.get_add_new_closest_dataframe(origins, origin_id_field, destinations, destination_id_field, closest_table, new_destination)

Wall time: 45.1 s


In [4]:
%%time
origin_df_remove = analysis.get_remove_existing_closest_dataframe(origins, origin_id_field, destinations, destination_id_field, closest_table, remove_destination_id)

Wall time: 42.2 s
